# Part 2: Specific Neural Architectures for NLP

**_Yuriy Guts_**

_UCU NLP Summer School, 2018_

## 2.3. Recurrent Neural Networks

As we have seen in the previous part, we can use a specific architectural component like a convolutional layer to make our model more suitable for local sequential patterns in the data (which are common for NLP problems).

While these representations are often an improvement over the traditional BoW model, their sensitivity to word order is mostly restricted to local patterns, and distant relationships in the sequence are still hard to capture.

Recurrent neural networks (RNN) [[Elman, 1990](https://onlinelibrary.wiley.com/doi/abs/10.1207/s15516709cog1402_1)] allow representing arbitrarily sized sequential inputs in fixed-size vectors, while paying attention to the structured properties of the inputs. RNNs, particularly ones with gated architectures such as the LSTM and the GRU, are very powerful at capturing statistical regularities in sequential inputs. They are arguably the strongest contribution of deep learning to the statistical NLP toolset.

### General Principles

The RNN is a function that takes as input an arbitrary-length, ordered sequence of $n$ $d_{in}$-dimensional vectors $x_{1:n} = x_1, x_2, ..., x_n (x_i \in \mathbb{R}^{d_{in}})$ and returns as output a single $d_{out}$-dimensional vector $y_n \in \mathbb{R}^{d_{out}}$:

$$
y_n = RNN(x_{1:n}) \\
x_i \in \mathbb{R}^{d_{in}} \quad y_n \in \mathbb{R}^{d_{out}}
$$

This implicitly defines an output vector $y_i$ for each prefix $x_{1:i}$ of the sequence $x_{1:n}$. We denote by $RNN^*$ the function returning this sequence:

$$
y_{1:n} = RNN^*(x_{1:n}) \\
y_i = RNN(x_{1:i}) \\
x_i \in \mathbb{R}^{d_{in}} \quad y_n \in \mathbb{R}^{d_{out}}
$$

The output vector $y_n$ can then be used for further prediction. For example, a model predicting the conditional probability of an event $e$ given the sequence $x_{1:n}$ can be defined as:

$$
p(e = j \: | \: x_{1:n}) = softmax(RNN(x_{1:n}) \cdot W + b)_{[j]}
$$

The $RNN$ function provides a framework for conditioning on the **entire history** $x_1, ..., x_i$ without resorting to Markov assumption which is traditionally used for modeling sequences.

The RNN is defined recursively, by means of two functions:

1. The state update function $R$, taking as input a state vector $s_{i-1}$ and an input vector $x_i$ and returning a new state vector $s_i$.

2. The output function $O$ which maps the state vector $s_i$ to an output vector $y_i$.

The base of the recursion is the initial state vector $s_0$ which is also an input to the RNN. For brevity, we often omit the initial vector $s_0$ or assume it is the zero vector.

The functions $R$ and $O$ are the same across the sequence positions but the RNN keeps track of the state through the state vector $s_i$ that is being passed across invocations of $R$.

Graphically, we can represent the RNN as follows:
<img src="rnn-cell-compact.png" width="300">

For a finite-sized input sequence (and all input sequences we deal with are finite) one can unroll the recursion, resulting in the following structure:

<img src="rnn-cell-unrolled.png">

Note that the value of $s_i$ (and therefore $y_i$) is based on the entire input $x_1, ..., x_n$. For example:

$$
\begin{equation}
\begin{split}
s_4 &= R(s_3, x_4) \\
    &= R(\overbrace{R(s_2, x_3)}^{s_3}, x_4) \\
    &= R(R(\overbrace{R(s_1, x_2)}^{s_2}, x_3), x_4) \\
    &= R(R(R(\overbrace{R(s_0, x_1)}^{s_1}, x_2), x_3), x_4)
\end{split}
\end{equation}
$$

The job of the network training is to set the parameters $\theta$ of $R$ and $O$ such that the state conveys useful information for the task we are trying to solve. Remember, like with the CNN architectures in the previous module, we are interested in providing useful task-specific features for the downstream layers of our neural model. The supervision signal is not applied to the RNN directly, but through the larger network.

it is easy to see that an unrolled RNN is just a very deep neural network (or rather, a very large computation graph with somewhat complex nodes), in which the same parameters are shared across many parts of the computation, and additional input is added at various layers. To train an RNN network, then, all we need to do is to create the unrolled computation graph for a given input sequence, add a loss node to the unrolled graph, and then use the backpropagation algorithm to compute the gradients with respect to that loss.

This procedure is referred to in the RNN literature as **backpropagation through time** (BPTT).

### Common RNN Usage Patterns

#### _Acceptor_

One option of using RNNs is to apply the supervision signal only at the final output vector $y_n$. We feed the entire sequence, observe the final state, and then decide on an outcome. For example, consider the task of detecting the language given a sequence of characters.

<img src="rnn-acceptor.png">

#### _Encoder_

An encoder is used similarly to the acceptor, meaning that we apply supervision only after the final output vector, $y_n$. However, unlike the acceptor, the final vector is treated only as one of the sources of information for the predictive model, and is used as an additional information together with other features.

For example, consider the task of detecting abusive content on a website with user-generated content. You may want to use the text of the post together with other features like the age of the user's account, previous interaction history and so on.

#### _Transducer_

Yet another option is to make the RNN produce an output $\hat{t}_i$ for each input it reads in. Modeled this way, we can compute a local loss signal $L_{local}(\hat{t}_i, t_i)$ for each of the intermediate outputs based on a true label $t_i$.

A very natural use case for the transduction setup is for language modeling, in which the sequence of words $x_{1:i}$ is used to predict a distribution over the next word $x_{i+1}$.

<img src="rnn-transducer.png">

#### _Bidirectional RNNs_

For certain tasks like machine translation or sequence tagging, the _future_ elements of the sequence may be just as helpful as the past ones. An elaboration of an RNN called a **bidirectional-RNN (biRNN)** defines two sets of functions: one for the forward pass $(R^f; O^f)$, another for the backward pass $(R^b; O^b)$. Much like the RNN relaxes the Markov assumption and allows looking arbitrarily back into the past, the biRNN relaxes the fixed window size assumption, allowing to look arbitrarily far at both the past and the future within the sequence.

<img src="rnn-bidirectional.png">

### Gated Architectures 

The "vanilla" RNN architecture described above is hard to train effectively because of the vanishing gradients problem [[Pascanu, Mikolov, Bengio; 2012](https://arxiv.org/abs/1211.5063)]. Error signals (gradients) in later steps in the sequence diminish quickly in the backpropagation process, and do not reach earlier input signals, making it hard for the RNN to capture long-range dependencies.

Gating-based architectures, such as the LSTM [[Hochreiter and Schmidhuber, 1997](http://www.bioinf.jku.at/publications/older/2604.pdf)] and the GRU [[Cho et al., 2014](https://arxiv.org/abs/1406.1078)] are designed to solve this deficiency.

Consider the RNN as ageneral-purpose computing device, where the state $s_i$ represents a finite memory. Each application of the function $R$ reads in an input $x_{i+1}$, reads in the current memory $s_i$, operates on them in some way, and writes the result into memory, resulting in a new memory state $s_{i+1}$. Viewed this way, an apparent problem with the RNN architecture is that the memory access is not controlled. At each step of the computation, the entire memory state is read, and the entire memory state is written.

How does one provide more controlled memory access? Consider a binary vector $g \in \{0, 1\}^n$. Such a vector can act as a **gate** for controlling access to $n$-dimensional vectors, using the Hadamard (element-wise) product $x \odot g$:

<img src="rnn-gate.png" width="450">

The gating mechanism described above can serve as a building block in our RNN: gate vectors can be used to control access to the memory state $s_i$.

However, we are still missing two important (and related) components: the gates should not be static, but be controlled by the current memory state and the input, and their behavior should be learned. This introduced an obstacle, as learning in our framework entails being differentiable (because of the backpropagation algorithm) and the binary 0-1 values used in the gates are not differentiable.

A solution to the above problem is to approximate the hard gating mechanism with a soft — but differentiable — gating mechanism. To achieve these differentiable gates, we replace the requirement that $g \in \{0, 1\}^n$ and allow arbitrary real numbers, $g' \in \mathbb{R}^n$, which are then pass through a sigmoid function $\sigma(g')$. This bounds the value in the range $(0,1)$, with most values near the borders. When using the gate $\sigma(g') \odot x$, indices in $x$ corresponding to near-one values in $\sigma(g')$ are allowed to pass, while those corresponding to near-zero values are blocked. 

The gate values can then be conditioned on the input and the current memory, and trained using a gradient-based method to perform a desired behavior. This controllable gating mechanism is the basis of the LSTM and the GRU architectures, to be defined next: at each time step, differentiable gating mechanisms decide which parts of the inputs will be written to memory, and which parts of memory will be overwritten (forgotten).

#### _LSTM_

The Long Short-Term Memory (LSTM) architecture was designed to solve the vanishing gradients problem, and is the first to introduce the gating mechanism. The LSTM architecture explicitly splits the state vector $s_i$ into two halves, where one half is treated as "memory cells" and the other is working memory. The memory cells are designed to preserve the memory, and also the error gradients, across time, and are controlled through differentiable gating components. At each input state, a gate is used to decide how much of the new input should be written to the memory cell, and how much of the current content of the memory cell should be forgotten.

<img src="lstm-cell.png" width="600">

Mathematically, the LSTM architecture is defined as:

\begin{equation}
\begin{split}
s_j = R_{LSTM}(s_{j-1}, x_j) &= [c_j; h_j] \\
                         c_j &= f \odot c_{j-1} + i \odot z \\
                         h_j &= o \odot \text{tanh}(c_j) \\
                           i &= \sigma(x_j W^{xi} + h_{j - 1} W^{hi}) \\
                           f &= \sigma(x_j W^{xf} + h_{j - 1} W^{hf}) \\
                           o &= \sigma(x_j W^{xo} + h_{j - 1} W^{ho}) \\
                           z &= \text{tanh}(x_j W^{xz} + h_{j - 1} W^{hz})
\end{split}
\end{equation}

$$
y_j = O_{LSTM}(s_j) = h_j \\
s_j \in \mathbb{R}^{2 \cdot d_h} \quad x_i \in \mathbb{R}^{d_x} \quad c_j, h_j, i, f, o, z \in \mathbb{R}^{d_h} \quad W^{xo} \in \mathbb{R}^{d_x \times d_h}
$$

The state at time $j$ is composed of two vectors, $c_j$ and $h_j$, where $c_j$ is the memory component and $h_j$ is the hidden state component. There are three gates, $i$, $f$, and $o$, controlling for **i**nput, **f**orget, and **o**utput. The gate values are computed based on linear combinations of the current input $x_j$ and the previous state $h_{j-1}$, passed through a sigmoid activation function. An update candidate $z$ is computed as a linear combination of $x_j$ and $h_{j-1}$, passed through a tanh activation function. The memory $c_j$ is then updated: the forget gate controls how much of the previous memory to keep ($f \odot c_{j-1}$), and the input gate controls how much of the proposed update to keep ($i \odot z$). Finally, the value of $h_j$ (which is also the output $y_j$) is determined based on the content of the memory $c_j$, passed through a tanh nonlinearity and controlled by the output gate. The gating mechanisms allow for gradients related to the memory part $c_j$ to stay high across very long time ranges.

LSTMs are currently the most successful type of RNN architecture, and they are responsible for many state-of-the-art sequence modeling results. The main competitor of the LSTM-RNN is the GRU, to be discussed below.

#### _GRU_

The LSTM architecture is very effective, but also quite complicated. The complexity of the system makes it hard to analyze, and also computationally expensive to work with. The gated recurrent unit (GRU) was recently introduced by Cho et al. [2014] as an alternative to the LSTM. It was subsequently shown by Chung et al. [2014] to perform comparably to the LSTM on several (non-textual) datasets.
Like the LSTM, the GRU is also based on a gating mechanism, but with substantially fewer gates and without a separate memory component.

<img src="gru-cell.png" width="350">

\begin{equation}
\begin{split}
s_j = R_{GRU}(s_{j-1}, x_j) &= (1 - z) \odot s_{j-1} + z \odot \tilde{s_j} \\
                          z &= \sigma(x_j W^{xz} + s_{j-1} W^{sz}) \\
                          r &= \sigma(x_j W^{xr} + s_{j-1} W^{sr}) \\
                \tilde{s_j} &= \text{tanh}(x_j W^{xs} + (r \odot s_{j-1}) W^{sg})
\end{split}
\end{equation}

$$
y_j = O_{GRU}(s_j) = h_j \\
s_j, \tilde{s_j} \in \mathbb{R}^{d_s} \quad x_i \in \mathbb{R}^{d_x} \quad z, r \in \mathbb{R}^{d_s} \quad W^{x*} \in \mathbb{R}^{d_x \times d_s} \quad W^{x*} \in \mathbb{R}^{d_s \times d_s}
$$

One gate ($r$) is used to control access to the previous state $s_{j-1}$ and compute a proposed up- date $\tilde{s_j}$. The updated state $s$ (which also serves as the output $y$) is then determined based on an interpolation of the previous state s and the proposal $\tilde{s_j}$, where the proportions of the interpolation are controlled using the gate $z$.

The GRU was shown to be effective in language modeling and machine translation. However, the jury is still out between the GRU, the LSTM and possible alternative RNN architectures, and the subject is actively researched. 

#### _Beyond differentiable gates_

Beyond differential gates the gating mechanism is an example of adapting concepts from the theory of computation (memory access, logical gates) into differentiable — and hence gradient-trainable — systems. There is considerable research interest in creating neural network architectures to simulate and implement further computational mechanisms, allowing better and more fine grained control.

One such example is the work on a differentiable stack [Grefenstette et al., 2015] in which a stack structure with push and pop operations is controlled using an end-to-end differentiable network, and the neural turing machine [Graves et al., 2014] which allows read and write access to content-addressable memory, again, in a differentiable system. While these efforts are yet to result in robust and general-purpose architectures that can be used in non-toy language processing applications, they are well worth keeping an eye on.

### References and Further Reading

1. [Yoav Goldberg - Neural Network Models for NLP [Morgan & Claypool, 2017]](https://www.morganclaypool.com/doi/abs/10.2200/S00762ED1V01Y201703HLT037).
2. [Jeffrey L. Elman - Finding Structure in Time [1990]](https://onlinelibrary.wiley.com/doi/abs/10.1207/s15516709cog1402_1) (RNN paper).
3. [Razvan Pascanu, Tomas Mikolov, Yoshua Bengio - On the difficulty of training Recurrent Neural Networks [2012]](https://arxiv.org/abs/1211.5063).
4. [Sepp Hochreiter, Jurgen Schmidhuber - Long Short-Term Memory [1997]](http://www.bioinf.jku.at/publications/older/2604.pdf) (LSTM paper).
5. [Kyunghyun Cho et al. - 
Learning Phrase Representations using RNN Encoder-Decoder for Statistical Machine Translation [2014]](https://arxiv.org/abs/1406.1078) (GRU paper).
6. [Chris Olah - Understanding LSTM Networks](http://colah.github.io/posts/2015-08-Understanding-LSTMs/).